In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

# get genotype data

In [ ]:
#get first round genotyping
geno = pd.read_csv('/home/lindb/teakettle/genemarker/results.tsv',header=0,index_col=None,sep='\t')
cols = [col for col in sorted(geno.columns) if 'samp' not in col]
geno = geno[['samp']+cols]
geno.replace(np.nan,"NA",inplace=True)
#geno = geno.loc[np.where(geno['samp']!='NA')[0].tolist(),:]
geno.head()

In [ ]:
#make sure the second allele is there
nucs = ['rps2','rps39','rps50','rps12']
for row in geno.index:
    for nuc in nucs:
        if geno.loc[row,"%s_2" % nuc] == "NA" and geno.loc[row,nuc] != "NA":
            print 'homogenizing %s\'s %s' % (geno.loc[row,'samp'],nuc)
            geno.loc[row,"%s_2" % nuc] = geno.loc[row,nuc]

In [ ]:
geno.to_csv('/home/lindb/teakettle/data/geno.txt',index=False,sep='\t')

In [ ]:
len(geno.index)

In [ ]:
#get second round genotyping
filE = '/home/lindb/teakettle/genemarker/results_round_2.tsv'
geno2 = pd.read_csv(filE,index_col=None,sep='\t')
geno2 = geno2[['samp']+cols]
geno2.replace(np.nan,"NA",inplace=True)
geno2 = geno2.loc[np.where(geno2['samp']!='NA')[0].tolist(),:]
geno2.head()

In [ ]:
#make sure the second alleles are there for homozygotes
nucs = ['rps2','rps39','rps50','rps12']
for row in geno2.index:
    for nuc in nucs:
        if geno2.loc[row,"%s_2" % nuc] == "NA" and geno2.loc[row,nuc] != "NA":
            print 'homogenizing %s\'s %s' % (geno2.loc[row,'samp'],nuc)
            geno2.loc[row,"%s_2" % nuc] = geno2.loc[row,nuc]

In [ ]:
geno2.to_csv('/home/lindb/teakettle/data/geno2.txt',header=True,index=False,sep='\t')

In [ ]:
len(geno2.index)

In [ ]:
samps = uni(geno2['samp'].tolist() + geno['samp'].tolist())
samps = [samp for samp in samps if samp != 'NA']
len(samps)

In [ ]:
#make a blank df to put in values
dsamps = sorted(samps + samps)
g = pd.DataFrame(index=dsamps,columns=geno2.columns)
g['samp'] = g.index.tolist()
g.index = np.arange(len(g.index))
g.replace(np.nan,'NA',inplace=True)
g.head()

In [ ]:
# fill in blank df
dfs = [geno,geno2]
for i,df in enumerate(dfs):
    for row in df.index:
        w = np.where(g['samp']==df.loc[row,'samp'])[0].tolist()
        for col in df.columns[1:]:
            if not g.loc[w[i],col] == "NA": #overwriting is bad, mkay
                print 'crap'
            else:
                if not df.loc[row,col] == "NA":
                    g.loc[w[i],col] = int(np.round(df.loc[row,col]))

In [ ]:
g.replace("NA",0,inplace=True)
g.head()

In [ ]:
filE = '/home/lindb/teakettle/data/full_geno.txt'
g.to_csv(filE,header=True,index=False,sep='\t')

In [ ]:
filE = '/home/lindb/teakettle/data/full_geno.txt'
g = pd.read_csv(filE, header=0,index_col=None,sep='\t')
g.head()

In [ ]:
loci = sorted(nucs + ['pc10','pt71936','pt87268'])
loci

In [ ]:
#look at missing data
c = Counter()
x = ""
for samp in uni(g['samp']):
    df = g.loc[np.where(g['samp']==samp)[0].tolist(),:]
    assert luni(df['samp']) == 1
    i,j = (df.index[0],df.index[1])
    miss = df.loc[i,loci].tolist().count(0)
    if miss == 0:
        c[str(miss)] += 1
    else:
        for l in loci:
            if df.loc[i,l] == 0:
                if not df.loc[j,l] == 0:
                    miss = miss - 1
#                     x = 'break'
        c[str(miss)] += 1
#     if x == 'break':
#         break

In [ ]:
68+22+116+90

In [ ]:
296/96

In [ ]:
blanks = [samp for samp in g['samp'] if 'blank' in samp]
b = g.loc[[np.where(g['samp'] == x)[0].tolist()[0] for x in g['samp'].tolist() if x in blanks],:]
b.head()

In [ ]:
luni(g.index)

In [ ]:
g.index[1:10]

# get tree location data from master data, add X Y to geno df

In [ ]:
#get tree location data from master data, add X Y to geno df
t = pd.read_csv('/home/lindb/teakettle/data/trees.txt',sep='\t',index_col=None)
t.replace("TRUE",True,inplace=True)
t.replace("FALSE",False,inplace=True)
t.replace('True',True,inplace=True)
t.replace('False',False,inplace=True)
t = t.loc[t['sampled'] != False,:]
t.index = np.arange(len(t.index))
t.head()

In [ ]:
def convert(s):
    num,plot = (s[:-3],s[-3:])
    if 'new' in num:
        num,new = (num[:-3],num[-3:])
        while len(num) < 3:
            num = '0%s' % num
    else:
        new = ""
    samp = "%s_%s%s" % (plot,new,num)
    return(samp)

In [ ]:
#convert sample name to convention in genotype data
t['converted'] = t['COMBO'].apply(lambda x: convert(x))

In [ ]:
grid = ['converted','seedling','x','y','z','xutm','yutm','zutm','dbh_11','dbh_04','dbh_13']
info = t.loc[:,grid]
info.head()

In [ ]:
#make sure either True or MD (missing data)
uni(t['sampled'])

# fix a few sample names in g

In [ ]:
msamps = []
for s in uni(g['samp']):
    if not s in t['converted'].tolist():
        if not 'blank' in s:
            msamps.append(s)
            print s

In [ ]:
len(msamps)

In [ ]:
len(g.index)

In [ ]:
#convert names if necessary, keep track of weird samps (may have forgotten to put sampled = True in t.DataFrame)
skips = []
count = 0
track = []
changed = []
for row in g.index:
    sold = ""
    s = g.loc[row,'samp']
    count += 1
    if s in msamps:
        sold = s
        if 'new' in s:
            while s not in t['converted'].tolist():
                plt_new,num = (s[:7],s[7:])
                s = "".join([plt_new,"0%s" % num])
                if s in t['converted'].tolist():
                    print "converted %s to %s" % (sold,s)
                    changed.append(sold)
                    g.loc[row,'samp'] = s
        elif '0' in s:
            plot,num = s.split("_")
            if not num.startswith("0"):
                print "skipped %s" % sold
                skips.append(sold)
                continue
            while s not in t['converted'].tolist():
                num = num[1:]
                s = "_".join([plot,num])
                if s in t['converted'].tolist():
                    print "found %s to be %s" % (sold,s)
                    changed.append(sold)
                    g.loc[row,'samp'] = s
        else:
            print "skipped %s" % sold
            skips.append(sold)
    track.append(s)
count

In [ ]:
#make sure I'm not screwing something up
for row in g.index:
    s = g.loc[row,'samp']
    if s in msamps:
        if s not in changed:
            if s not in skips:
                print "dammit %s!" % s

In [ ]:
uni(skips)

In [ ]:
uni(skips)

In [ ]:
['bc3_3980', #sampled = F
 'uc2_3141',    #not in plot data
 'uc2_5185',    #not in plot data
 'uc2_9554',    #not in plot data 
 'uc2_9592',    #not in plot data 
 'uc2_9920', #change to 9220 in g
 'un1_2749',    #not in plot data
 'un2_3126',    #not in data
 'un3_1542', #sampled = F
 'un3_2588',    #not in data
 'us3_1087', #marked as abco in data
 'us3_1752']    #not in data

In [ ]:
fix = ['bc3_3980',
       'uc2_9920',
       'un1_2749',
       'un2_3126',
       'un3_1542',
       'us3_1087',]

In [ ]:
#fix errors and redo above (leaving the fix scripts here)
tree = pd.read_csv('/home/lindb/teakettle/data/trees.txt',sep='\t',index_col=0)
tree.index = np.arange(len(tree.index))
tree['converted'] = tree['COMBO'].apply(lambda x: convert(x))
tree.head()

In [ ]:
np.where(tree['converted'] == 'bc3_3980')[0][0]

In [ ]:
tree.loc[6285,['COMBO','converted','sampled']]

In [ ]:
tree.loc[6285,'sampled'] = 'True'

In [ ]:
np.where(g['samp'] == 'uc2_9920')

In [ ]:
g.loc[602,'samp'] = 'uc2_9220' 
g.loc[603,'samp'] = 'uc2_9220' #I saved g to overwrite

In [ ]:
np.where(tree['converted'] == 'un3_1542')[0][0]

In [ ]:
tree.loc[32670,['species','converted','sampled']]

In [ ]:
tree.loc[32670,'sampled'] = 'True'

In [ ]:
np.where(tree['converted'] == 'us3_1087')[0][0]

In [ ]:
tree.loc[38760,['species','converted','sampled']]

In [ ]:
tree.loc[38760,['species']] = 'pila'
tree.loc[38760,['sampled']] = 'True'

In [ ]:
filE = '/home/lindb/teakettle/data/trees.txt'
tree.to_csv(filE,header=True,index=False,sep='\t')

# combine dfs

In [ ]:
# g.index = g['samp']
g.index = g['samp'].tolist()
g.head()

In [ ]:
info.index = info['converted']
info.head()

In [ ]:
info.loc[info.index[0],'dbh_04']

In [ ]:
for row in info.index:
    if info.loc[row,'seedling'] == False:
        if math.isnan(info.loc[row,'dbh_11']):
            if math.isnan(info.loc[row,'dbh_04']) == False:
                info.loc[row,'dbh_11'] = info.loc[row,'dbh_04']*0.9699+5.1995 #lm(dbh_11 ~ dbh_04)
            else:
                info.loc[row,'dbh_11'] = 28.37125 #(median + mean)/2  -- 6 trees

In [ ]:
np.mean(info['dbh_11'])

In [ ]:
c = pd.merge(g,info,left_index=True,right_index=True)
c.head()

In [ ]:
ccols = ['samp',
 'x',
 'y',
 'dbh_11',
 'xutm',
 'yutm',
 'seedling',
 'rps12',
 'rps12_2',
 'rps2',
 'rps2_2',
 'rps39',
 'rps39_2',
 'rps50',
 'rps50_2',
 'pc10',
 'pt71936',
 'pt87268']
c = c[ccols]
c.head()

In [ ]:
c.loc['bc1_new007','dbh_11']

In [ ]:
len(c.index)

In [ ]:
c.index = np.arange(len(c.index))

In [ ]:
c.loc[row,'seedling']

In [ ]:
c.replace(np.nan,'NA',inplace=True)
DIR = '/home/lindb/teakettle/data/moran'
if not op.exists(DIR):
    os.makedirs(DIR)
c.to_csv(op.join(DIR,'all_data.txt'),header=True,index=False,sep='\t')

In [ ]:
len(c.index)

In [ ]:
len(g.index)

In [ ]:
len(set(uni(g['samp'])) - set(info['converted']))

In [ ]:
2610-2552 == 29*2 #difference in index length is due to blanks and samps not in plot data

In [ ]:
plots = []
for row in c.index:
    plot = c.loc[row,'samp'][0].split("_")[0]
    if not plot in plots:
        plots.append(plot)
len(plots),plots

In [ ]:
c.loc[row,'samp'][0]

# to do

1. compare vals to dyer's machine
1. make sure no name samples aren't from bleedthrough
1. remove "" samples and 'blank's

In [ ]:
files = sorted(ls('/home/lindb/teakettle/data/moran/amat_files2/'))
len(files)

In [ ]:
files = sorted(ls('/home/lindb/teakettle/data/moran/amat_files2/'),reverse=True)
len(files)

In [ ]:
files = sorted(ls('/home/lindb/teakettle/data/moran/amat_files2/'),reverse=True)
len(files)

In [ ]:
files[0:10]

In [ ]:
count = 0
for k in np.arange(558):
    i = k+1
    for q in np.arange(558):
        count += 1
        if count % 1000 == 0:
            print count
        j = q+1
        text = '''
source('/home/lindb/imports.R')

#load dependencies
dname = '/home/lindb/teakettle/data/moran/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        #cat(sprintf ("loading %%s file",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

amat <- numeric(0)
for (k in 1:noff)
{
    a <- matrix(0,(nadult+1),(nadult+1)) 
    amat <- append(amat,list(a))
}
    
i <- %s
j <- %s
#print ("i=")
#print(i)
#print("j=")
#print(j)


if(i>0) #we want it to run no matter what
{
    print ("starting loop")
    if(i==j & i<(nadult+1)) #this is the diagonal, can't have same sampled mother and father
    {
        unimportant = 1+1 # amat is already 0 on the diagonal, save some time by not looping
#         for (k in 1:noff)
#         {
#             amat[[k]][i,j] <- 0   
#         }
    }
    if(i==j & i==(nadult+1)) #can have two out-of-plot parents
    { 
        nmat <- numeric(0)
        pg1 <- pgout 
        pg1.b <- pgout 
        pg2 <- pgout 
        pg2.b <- pgout
        #print ("starting loci loop")
        for(la in 1:nloci) 
        {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
            #at the la'th locus
            #print (la)
            nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) #returns a matrix with ncol = nrow = nallele @ la'th locus
            nmat <- append(nmat,list(nm))
        }
        #print ("starting offspring loop")
        for(k in 1:noff)
        {
            #print (k)
            pq <- 1 #incorrectly overwriting pq is bad, mkay
            og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the offspring's observed first and second alleles
            og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,]) #get any offspring's observed regenotyping results
            #print ("starting loci loop")
            for(la in 1:nloci)
            {
                #print (la)
                #om = probability of observing the given offspring genotype given all possible true genotypes
                om <- omat.func(og,og.b,la) 
            #pq = probability of observing the given offspring genotype given the observed parental genotypes
                pq <- pq*sum(nmat[[la]]*om)
                #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                if(pq==0) 
                {
                    break 
                }
            }
            #print ("writing amat")
            amat[[k]][i,j] <- pq #pq same as above, for the k'th offspring for the i'th and j'th parent
            amat[[k]][j,i] <- pq 
        }# end offspring loop
    }# end two-out-of-plot loop
    if(i!=j) #if not on the diagonal
    { #all other combos... 
        nmat <- numeric(0)
        if(i<(nadult+1)) #if posited parent one of those who were sampled
        {
            #gadult# == nrow=nadult ncol=nloci
            pg1 <- rbind(gadult[[1]][i,],gadult[[2]][i,])  #get first and second alleles for all loci
            pg1.b <- rbind(gadult.r[[1]][i,],gadult.r[[2]][i,]) #get first and second alleles for regenotyping
        } 
        if(j<(nadult+1)) #if posited parent one of those who were sampled
        {
            pg2 <- rbind(gadultF[[1]][j,],gadultF[[2]][j,])
            pg2.b <- rbind(gadultF.r[[1]][j,],gadultF.r[[2]][j,]) 
        }
        if(i==(nadult+1)) #if posited parent is not one of those who were sampled
        { 
            pg1 <- pgout 
            pg1.b <- pgout
        } 
        if(j==(nadult+1)) #if posited parent is not one of those who were sampled
        {
            pg2 <- pgout #put in all zeros
            pg2.b <- pgout #put in all zeros
        }
        #print ("starting loci loop")
        for(la in 1:nloci)
        {#Calculate probability of offspring genotype given parental genotypes using global allele frequencies
            #at the la'th locus
            print (la)
            nm <- nmat.func(pg1,pg2,pg1.b,pg2.b,la) 
            nmat <- append(nmat,list(nm))
        }
        #print ("starting offspring loop")
        for(k in 1:noff)
        { #pre-calc nmat at all loci for parent pair
            #print(k)
            pq <- 1 #incorrectly overwriting is still bad, mkay
            og <- rbind(goff[[1]][k,],goff[[2]][k,]) #get the k'th offspring's 1st and 2nd alleles @ all loci
            og.b <- rbind(goff.r[[1]][k,],goff.r[[2]][k,])
            #print ("starting loci loop")
            for(la in 1:nloci)
            {
                print (la)
                #om = probability of observing the given offspring genotype given all possible true genotypes
                om <- omat.func(og,og.b,la)
                #pq = probability of observing the given offspring genotype given the observed parental genotypes
                pq <- pq*sum(nmat[[la]]*om)
                if(pq==0) 
                {
                #if the posited parents are not the parents, no use in doing futher calculations @ other loci
                    break 
                }
            }
            amat[[k]][i,j] <- pq 
            amat[[k]][j,i] <- pq 
        }# end offspring loop
    }#end "all other combos" loop
    
#     track[i,j] <- 1
#     track[j,i] <- 1

} 

#print ("writing RDS files")
DIR = '/home/lindb/teakettle/data/amat_outfiles/'
amatfname = paste(DIR,'amat_%s_%s.RDS',sep="")
nmatfname = paste(DIR,'nmat_%s_%s.RDS',sep="")
#sprintf("writing %%s",amatfname)
saveRDS(amat,amatfname)
#sprintf("writing %%s",nmatfname)
saveRDS(nmat,nmatfname)

''' % (i,
       j,
       str(i).zfill(3),
       str(j).zfill(3),
       str(i).zfill(3),
       str(j).zfill(3)
      )
        DIR = '/home/lindb/teakettle/data/moran/amat_files2'
        if not op.exists(DIR):
            os.makedirs(DIR)
        filE = op.join(DIR,"amat_%s_%s.R" % (str(i).zfill(3),str(j).zfill(3)))
        with open(filE,'wb') as o:
            o.write("%s" % text)

# at some point go back and compare Dyer's machine to Cornell